[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/neuralangelo-colab/blob/main/neuralangelo_colab.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/neuralangelo
%cd /content/neuralangelo

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/neuralangelo/resolve/main/toy_example.MOV -d /content/test -o toy_example.MOV
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/neuralangelo/resolve/main/neus.zip -d /content/datasets/dtu -o neus.zip
%cd /content/datasets/dtu
!unzip -q neus.zip -d /content/datasets/dtu
%cd /content/neuralangelo
!python /content/neuralangelo/projects/neuralangelo/scripts/convert_dtu_to_json.py --dtu_path /content/datasets/dtu